# Sequential Decision Modeling

## Step 1: Narrative

## Step 2: Identify Core Metrics

* Maximize contribution.
* Demand for each day is unknown.
    * $\hat{W}_{t+1}=\hat{D}_{t+1}$: Demand for today.
    * $\hat{D} \sim \mathcal{N}(\mu = 50, \sigma^2 = 5)$
* Decision variable $x_t$: How many pounds of bacon we'll order at the end of the day. $t$
* Constants:
    * $p=25$
    * $c=15$
* Initial values:
    * $R_0=10$
* Policies:
    * $\theta \in [40,60]$

## Step 3: Mathematical Model

### State:

$$S_t=(R_t, \hat{D}_{t+1}, c, p, \mu, \sigma^2)$$

### Decision Variables:

$$X^\pi = \max(\theta - R_t, 0)$$

### Exogenous Informartion:

$$\hat{D}_{t+1} \sim \mathcal{N}(\mu = 50, \sigma^2 = 5)$$

### Transition Function:

$$S^M=\max(R_t+x_t-\hat{D}_{t+1}, 0)$$

### Objective Function:

$$\max_{\pi} \sum_{t=1}^T C(S_t,x_t) | S_0$$

$$C(S_t,x_t)=-c(x_t)+p(\min(R_t+x_t,\hat{D}_{t+1}))$$

## Step 4: Uncertainity Model

* Previously defined...

## Step 5: Designing Policies

$$\theta \in [40,60]$$

## Step 6: Evaluating Policies

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
def X_t(r_t: int, theta: int) -> int:
    return np.max([0, theta - r_t])

In [3]:
c = 15
p = 25
mu = 50
var = 5
r_0 = 10

theta_space = list(range(40, 61))

for theta in theta_space:
    contributions = []

    for _ in range(1000):
        scenarios = []
        r_t = r_0
    
        for i in range(1, 11):
            # Buy inventory
            x_t = X_t(r_t, theta)

            # Exogenous information
            D_t_1 = np.round(np.random.normal(mu, np.sqrt(var)))

            # Transition function
            r_t_1 = np.max([x_t + r_t - D_t_1, 0])

            contribution = -c * x_t + p * (np.min([x_t + r_t, D_t_1]))

            scenarios.append({
                "t": i,
                "r_t": r_t,
                "x_t": x_t,
                "D_t_1": D_t_1,
                "C": contribution,
            })

            r_t = r_t_1

        theta_df = pd.DataFrame.from_records(scenarios)

        contributions.append(theta_df.C.sum())

    print(f"Theta = {theta} -> {np.mean(contributions):.2f}")

Theta = 40 -> 4150.00
Theta = 41 -> 4250.00
Theta = 42 -> 4349.98
Theta = 43 -> 4449.86
Theta = 44 -> 4549.70
Theta = 45 -> 4648.86
Theta = 46 -> 4746.20
Theta = 47 -> 4839.91
Theta = 48 -> 4923.74
Theta = 49 -> 4995.72
Theta = 50 -> 5044.78
Theta = 51 -> 5079.06
Theta = 52 -> 5099.70
Theta = 53 -> 5090.73
Theta = 54 -> 5082.55
Theta = 55 -> 5067.95
Theta = 56 -> 5061.55
Theta = 57 -> 5047.18
Theta = 58 -> 5027.91
Theta = 59 -> 5017.35
Theta = 60 -> 5005.89
